In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import seaborn as sns

In [4]:
sns.set_context('poster', font_scale=1.25)

In [49]:
w=np.array([4./9.,1./9.,1./9.,1./9.,1./9.,1./36.,    
            1./36.,1./36.,1./36.]) # weights for directions
cx=np.array([0,1,0,-1,0,1,-1,-1,1]) # direction vector for the x direction
cy=np.array([0,0,1,0,-1,1,1,-1,-1]) # direction vector for the y direction
tau=1
cs=1/math.sqrt(3)
cs2 = cs**2
cs22 = 2*cs2
cssq = 2.0/9.0

w0 = 4./9.
w1 = 1./9.
w2 = 1./36.

In [50]:
viscosity = cs**2*(tau-0.5)
print 'Viscosity is:' , viscosity

Viscosity is: 0.166666666667


In [51]:
lx=400  # length of domain in the x direction
ly=400  # length of domain in the y direction

We know that

$\nu = \frac{\Delta x^2}{3 \Delta t}(w - \frac{1}{2})$

from my engineering book, or

$\nu = c_s^2(\tau-\frac{1}{2})=c_s^2(\frac{1}{w}-\frac{1}{2})$

We begin by calculating macrosocpic quantities. The initial conditions. Let's make constant density everywhere of one.

In [52]:
rho = np.ones((lx, ly))
u_applied=cs/100
u = u_applied*(np.ones((lx, ly)) + np.random.randn(lx,ly))
v= (u_applied/100.)*(np.ones((lx, ly)) + np.random.randn(lx,ly)) # initializing the vertical velocities

In [58]:
f=np.zeros((9,lx,ly)) # initializing f
feq = np.zeros((9, lx, ly))

We know that $\sum_i w_i f_i = \rho$, so

In [59]:
for j in range(w.shape[0]):
    f[j,:,:]=w[j]*rho

Let's just do this ourselves. Ok, so we have f0. We now need to stream and update the density array. 

In [60]:
# Taken from sauro succi's code. This will be super easy to put on the GPU.
ul = u/cs2
vl = v/cs2
uv = ul*vl
usq = u*u
vsq = v*v
sumsq  = (usq+vsq)/cs22
sumsq2 = sumsq*(1.-cs2)/cs2
u2 = usq/cssq 
v2 = vsq/cssq

feq[0, :, :] = w0*(1. - sumsq)

feq[1, :, :] = w1*(1. - sumsq  + u2 + ul)
feq[2, :, :] = w1*(1. - sumsq  + v2 + vl)
feq[3, :, :] = w1*(1. - sumsq  + u2 - ul)
feq[4, :, :] = w1*(1. - sumsq  + v2 - vl)
feq[5, :, :] = w2*(1. + sumsq2 + ul + vl + uv)
feq[6, :, :] = w2*(1. + sumsq2 - ul + vl - uv)
feq[7, :, :] = w2*(1. + sumsq2 - ul - vl + uv)
feq[8, :, :] = w2*(1. + sumsq2 + ul - vl - uv)